In [1]:
from simtk import unit, openmm
from simtk.openmm import app
from alchemy import AbsoluteAlchemicalFactory, AlchemicalState

import blues.utils as utils
import blues.ncmc as ncmc
import blues.ncmc_switching as blues_switching
from blues.smartdart import SmartDarting

import sys
import numpy as np
import mdtraj as md
from mdtraj.reporters import HDF5Reporter
from datetime import datetime
from optparse import OptionParser

In [2]:
# Define some constants
temperature = 300.0*unit.kelvin
friction = 1/unit.picosecond
dt = 0.002*unit.picoseconds
# set nc attributes
numIter = 10
nstepsNC = 10
nstepsMD = 50
#Defines ncmc move eqns for lambda peturbation of sterics/electrostatics
functions = { 'lambda_sterics' : 'step(0.199999-lambda) + step(lambda-0.2)*step(0.8-lambda)*abs(lambda-0.5)*1/0.3 + step(lambda-0.800001)',
            'lambda_electrostatics' : 'step(0.2-lambda)- 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)' }

In [3]:
# Initialize integrators
md_integrator = openmm.LangevinIntegrator(temperature, friction, dt)
alch_integrator = openmm.LangevinIntegrator(temperature, friction, dt)

In [4]:
# Obtain topologies/positions
coord_file = 'eqToluene.inpcrd'
top_file =   'eqToluene.prmtop'
prmtop = app.AmberPrmtopFile(top_file)
inpcrd = app.AmberInpcrdFile(coord_file)

In [5]:
# Generate OpenMM System
system = prmtop.createSystem(nonbondedMethod=app.PME,
                            nonbondedCutoff=1*unit.nanometer,
                            constraints=app.HBonds)

In [6]:
# Initailize MD Simulation
md_sim = app.Simulation(prmtop.topology,
                        system,
                        md_integrator)
md_sim.context.setPositions(inpcrd.positions)
md_sim.context.setVelocitiesToTemperature(temperature)
md_sim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

In [7]:
# Add reporters for MD simulation
md_sim.reporters.append(app.dcdreporter.DCDReporter('traj.dcd', nstepsMD))
md_sim.reporters.append(HDF5Reporter('traj.h5', nstepsMD))

In [8]:
# Get ligand atom indices
lig_atoms = utils.atomIndexfromTop('LIG', prmtop.topology)

In [9]:
#Initialize Alchemical Simulation
# performs alchemical corrections
# Reporter for NCMC moves
alch_sim = app.Simulation(prmtop.topology,
                          system,
                          alch_integrator)
alch_sim.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
# Generate Alchemical System
factory = AbsoluteAlchemicalFactory(system,
                                    ligand_atoms=lig_atoms,
                                    annihilate_sterics=True,
                                    annihilate_electrostatics=True)
alch_system = factory.createPerturbedSystem()

In [10]:
# Generate NC Integrator/Contexts
nc_integrator = blues_switching.NCMCVVAlchemicalIntegrator(temperature,
                                            alch_system,
                                            functions,
                                            nsteps=nstepsNC,
                                            direction='insert',
                                            timestep=0.001*unit.picoseconds,
                                            steps_per_propagation=1)
nc_context = openmm.Context(alch_system, nc_integrator)
nc_context.setPeriodicBoxVectors(*inpcrd.boxVectors)

In [11]:
# Initialize BLUES engine
blues_run = ncmc.SimNCMC(temperature=temperature, residueList=lig_atoms)

In [12]:
#Define NC move
rot_step = (nstepsNC/2) -1
nc_move = [[blues_run.rotationalMove, [rot_step]]]

In [13]:
# actually run
blues_run.get_particle_masses(system, residueList=lig_atoms)
blues_run.runSim(md_sim, nc_context, nc_integrator,
                alch_sim, movekey=nc_move,
                niter=numIter, nstepsNC=nstepsNC, nstepsMD=nstepsMD,
                alchemical_correction=True)

performing ncmc step
accCounter = 0
doing the move
before step 0.4
[[ 2.52080418  2.39190064  2.93688492]
 [ 2.51780038  2.31732836  3.05857699]
 [ 2.4286179   2.36248409  2.83382508]
 [ 2.43369092  2.21337781  3.07343845]
 [ 2.33666224  2.26012302  2.85853081]
 [ 2.33518231  2.18767085  2.97703183]
 [ 2.24230161  2.07593444  2.99255923]
 [ 2.5960074   2.46943097  2.92528607]
 [ 2.58705285  2.33764185  3.14056137]
 [ 2.44071271  2.41532308  2.73995397]
 [ 2.42882397  2.14910901  3.16191354]
 [ 2.26122777  2.2408782   2.78119859]
 [ 2.29592222  1.98852414  3.03122797]
 [ 2.16402648  2.10387687  3.0646232 ]
 [ 2.18252286  2.05470002  2.90445921]] rot_output
[ 2.23699999  2.16799998  2.977     ] [ 2.52080418  2.39190064  2.93688492]
[ 2.25999999  2.29299998  3.04200006] [ 2.51780038  2.31732836  3.05857699]
[ 2.33999991  2.10800004  2.90100002] [ 2.4286179   2.36248409  2.83382508]
[ 2.37599993  2.35899997  3.0250001 ] [ 2.43369092  2.21337781  3.07343845]
[ 2.46199989  2.17600012  2.8949

Quantity(value=array([[ 3.31110716,  1.1009351 ,  4.36174822],
       [ 3.27555275,  1.1216054 ,  4.45399666],
       [ 3.24273109,  1.03932738,  4.32015276],
       ..., 
       [ 1.82460368,  3.48960567,  0.76897037],
       [ 1.89344227,  3.45012903,  0.82249773],
       [ 1.75241458,  3.42685914,  0.77270257]]), unit=nanometer)